# CSAT ETL Notebook

This notebook outlines an ETL pipeline for analyzing Customer Satisfaction (CSAT) comments, from data loading and cleaning to text preprocessing, clustering, and saving results for further analysis in BigQuery & Looker Studio.

## Table of Contents
1. [Installation and Importing of Required Libraries](#Installation-and-Importing-of-Required-Libraries)
2. [Data Loading](#Data-Loading)
3. [Data Cleaning](#Data-Cleaning)
4. [Creating `conversations` table](#Creating-Conversations-Table)
5. [Creating `user_comments` table](#Creating-User-Comments)
6. [Text Preprocessing](#Text-Preprocessing)
    - [Stopword Removal](#Stopword-Removal)
    - [Noun Extraction](#Noun-Extraction)
7. [Embedding Generation](#Embedding-Generation)
8. [Clustering](#Clustering)
9. [Saving Results](#Saving-Results)


<a name="Installation-and-Importing-of-Required-Libraries"></a>
## 1. Installation and importing of required libraries

In [106]:
!pip install openpyxl
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install nltk
!pip install spacy
!pip install scikit-learn
!pip install sentence-transformers
!pip install tqdm

# NLTK specific downloads
import nltk
nltk.download('stopwords')

# spaCy specific downloads for the German model
!python -m spacy download de_core_news_sm

In [ ]:
import ast
import re

import nltk
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import spacy
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

<a name="Data-Loading"></a>
## 2. Data Loading
This section covers loading the CSAT comments from an Excel file into a pandas DataFrame for processing. The file path is specified, and the data is loaded using `pd.read_excel`.


In [74]:
file_path = 'csat_updated.xlsx'

# Load the Excel file into a pandas DataFrame
source_df = pd.read_excel(file_path)

<a name="Data-Cleaning"></a>
## 3. Data Cleaning
In this section, we clean the 'Subject' column by removing prefixes (e.g., "RE:", "FW:") and newlines.

In [75]:
def remove_re_prefix(subject):
    # Convert subject to string if it's not None/NaN
    if pd.isnull(subject):
        return subject
    subject = str(subject)  # Ensure subject is treated as a string
    
    # Remove newlines
    subject = subject.replace('\n', ' ').replace('\r', '')
    
    prefixes = ['RE: ', 'Re: ', 'RE:', 'Re:', 're: ', 're:', 'Re: FW: ', 'Re: FW:']
    for prefix in prefixes:
        if subject.startswith(prefix):
            subject = subject[len(prefix):].strip()
            break
    return subject

source_df['Subject'] = source_df['Subject'].apply(remove_re_prefix)


<a name="Creating-Conversations-Table"></a>
## 4. Creating `conversations` table
Here, the columns `Ratings` and `Rating comments` are removed since the source data will be normalised and the rating data will be moved to the table `user_comments`. The remaining column names are modified for consistency and ease of access. This includes renaming columns to lower case, removing spaces, periods, parentheses, and commas. The resulting table is called `conversations`.


In [77]:
conversations = source_df.drop(['Ratings', 'Rating comments'], axis=1)

# Adjust the column names: make lowercase, replace spaces and periods with underscores, remove parentheses
conversations.columns = (
    conversations.columns.str.lower()
    .str.replace('conversation #', 'conversation_id')
    .str.replace(' ', '_')
    .str.replace('.', '', regex=False)
    .str.replace('(', '', regex=False)
    .str.replace(')', '', regex=False)
    .str.replace(',', '', regex=False)
)

Index(['id', 'conversation_id', 'type', 'status', 'mailbox', 'thread_count',
       'subject', 'created_at', 'last_modified', 'closed_at',
       'first_response_time_office_hours_seconds', 'avg_response_time_seconds',
       'avg_response_time_office_hours_seconds', 'replies_sent',
       'handle_time_seconds', 'resolution_time_seconds',
       'resolution_time_office_hours_seconds'],
      dtype='object')
conversations saved to 'conversations.csv'


<a name="Creating-User-Comments"></a>
## 5. Creating `user_comments` table
A one-to-many relationship is identified in the source data, i.e a single conversation can have multiple ratings. Therefore, the user ratings are separated from the conversations and then merged into a table called `user_comments`

In [78]:
source_df['Rating comments'] = source_df['Rating comments'].apply(lambda x: 'None' if pd.isna(x) else x)

expanded_data = []

for index, row in source_df.iterrows():
    conv_id = row['Conversation #']
    ratings = row['Ratings'].split(',')
    comments_str = row['Rating comments']

    if comments_str not in [None, 'None']:
        try:
            comments = ast.literal_eval('[' + comments_str + ']')
        except (ValueError, SyntaxError):
            comments = [comments_str]
    else:
        comments = [None] * len(ratings)
    
    if len(comments) < len(ratings):
        # Ensure the last rating of the same type receives the comment
        adjusted_comments = [None] * (len(ratings) - len(comments)) + comments
    else:
        adjusted_comments = comments

    for rating, comment in zip(ratings, adjusted_comments):
        expanded_data.append({
            'conversation_id': conv_id,
            'rating': rating.strip(),
            'rating_comment': comment if comment != 'None' else None
        })

user_comments = pd.DataFrame(expanded_data)


   conversation_id rating                                     rating_comment
0           129789  great  Ganz herzlichen Dank, für die entgegenkommende...
1           126849  great                                               None
2           128766  great                                               None
3           130018  great                                               None
4           130018   okay                                               None
5           130665  great                    Vielen lieben Dank für die Info
6           130533  great                                               None
7           130632  great       Vielen Dank für die nette Antwort. LG Monika
8           130239  great                                               None
9           110248  great                                               None


<a name="Text-Preprocessing"></a>
## 6. Text Preprocessing
Text preprocessing steps include converting comments to lowercase, removing special characters, stopwords, and extracting nouns. This is facilitated by libraries such as `nltk` for stopwords removal and `spacy` for noun extraction.


In [79]:
def clean_rating_comment(comment):
    if pd.isnull(comment):
        return None  # Return None if comment is NaN or None
    comment = str(comment)  # Ensure comment is treated as a string
    # Remove newlines
    comment = comment.replace("\n", " ").replace("\r", " ")
    # Remove "None," or ",None" patterns and extra whitespaces
    comment = comment.replace("None,", "").replace(",None", "")
    # Trim leading and trailing whitespaces and remove surrounding double quotes
    comment = comment.strip().strip('"')
    return comment

# Apply the cleaning function to the 'rating_comment' column
user_comments['rating_comment'] = user_comments['rating_comment'].apply(clean_rating_comment)


In [107]:
# Ensure NLTK stopwords are downloaded
nltk.download('stopwords')

# Load German stopwords from NLTK
german_stopwords = set(stopwords.words('german'))

def clean_and_remove_stopwords(text):
    # Check if text is None or empty
    if not text:
        return None
    # Convert text to lowercase and tokenize by word characters, ignoring special characters
    words = re.findall(r'\w+', text.lower())
    # Remove stopwords and numeric strings
    filtered_words = [word for word in words if word not in german_stopwords and not word.isdigit()]
    return filtered_words

# Apply the function to 'rating_comment'
user_comments['words'] = user_comments['rating_comment'].apply(clean_and_remove_stopwords)

print(user_comments['words'].head())


0    [ganz, herzlichen, dank, entgegenkommende, sch...
1                                                 None
2                                                 None
3                                                 None
4                                                 None
Name: words, dtype: object


In [87]:
# Load the German language model
nlp = spacy.load("de_core_news_sm")

def extract_nouns(text):
    if not text:
        return []
    # Process the text with spaCy
    doc = nlp(text)
    # Extract nouns
    nouns = [token.text.lower() for token in doc if token.pos_ == "NOUN"]
    return nouns

user_comments['nouns'] = user_comments['rating_comment'].apply(extract_nouns)

print(user_comments[['rating_comment', 'nouns']].head())


                                      rating_comment  \
0  Ganz herzlichen Dank, für die entgegenkommende...   
1                                               None   
2                                               None   
3                                               None   
4                                               None   

                                 nouns  
0  [dank, lösung, super, kundendienst]  
1                                   []  
2                                   []  
3                                   []  
4                                   []  


<a name="Embedding-Generation"></a>
## 7. Embedding Generation
Embeddings are generated for each comment using the `SentenceTransformer` library. These embeddings capture the semantic meaning of the comments and are used for clustering. The embedding model is multilingual, allowing similar sentences in different languages to be grouped together.


In [82]:
# Initialize the embedder
embedder = SentenceTransformer("distiluse-base-multilingual-cased")

# Initialize a list to hold the embeddings
rating_embeddings = []

# Wrap the loop with tqdm for a progress bar
for comment in tqdm(user_comments['rating_comment'], desc="Generating Embeddings"):
    if comment is not None:
        # Generate the embedding for the comment
        embedding = embedder.encode([comment], show_progress_bar=False)
        rating_embeddings.append(embedding[0])
    else:
        # Append None for comments that are None
        rating_embeddings.append(None)

# Add the embeddings as a new column in the user_comments DataFrame
user_comments['rating_embedding'] = rating_embeddings

user_comments.head()


Generating Embeddings: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 9057/9057 [00:45<00:00, 198.87it/s]


,conversation_id,rating,rating_comment,words,nouns,rating_embedding
0,129789,great,"Ganz herzlichen Dank, für die entgegenkommende...","[ganz, herzlichen, dank, entgegenkommende, sch...","[dank, lösung, super, kundendienst]","[0.0013463228, -0.01692898, 0.0082012545, -0.0..."
1,126849,great,None,None,[],None
2,128766,great,None,None,[],None
3,130018,great,None,None,[],None
4,130018,okay,None,None,[],None


<a name="Clustering"></a>
## 8. Clustering
Using the embeddings, clustering is performed to group similar comments together. This helps identify common themes or issues within the feedback. The `KMeans` algorithm from `sklearn.cluster` is used for this purpose. Here, 10 clusters are created for each of the three rating categories.


In [83]:
# Initialize an empty DataFrame for csat_impactors
csat_impactors = pd.DataFrame(columns=['main_sentence', 'conversation_id', 'rating', 'cluster_id', 'type', 'id'])
csat_impactors_list = []

for rating in ['bad', 'okay', 'great']:
    # Filter for current rating and non-None embeddings
    filtered_df = user_comments[(user_comments['rating'] == rating) & (user_comments['rating_embedding'].notna())].copy()  # Ensure to work on a copy
    
    if filtered_df.empty:
        continue  # Skip if no comments for this rating
    
    embeddings = np.array(list(filtered_df['rating_embedding']))
    
    # Perform clustering
    num_clusters = 10  # Adjust as necessary
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans.fit(embeddings)
    
    # Assign cluster labels
    filtered_df['cluster_id'] = ['{}_{}'.format(rating, label+1) for label in kmeans.labels_]
    
    # Append updated rows back to user_comments
    for index, row in filtered_df.iterrows():
        user_comments.at[index, 'cluster_id'] = row['cluster_id']
        user_comments.at[index, 'local_cluster_id'] = row['cluster_id'].split("_")[1]
    
    # Process for csat_impactors
    for cluster_id in set(filtered_df['cluster_id']):
        cluster_df = filtered_df[filtered_df['cluster_id'] == cluster_id]
        cluster_embeddings = np.array(list(cluster_df['rating_embedding']))
        cluster_center = kmeans.cluster_centers_[int(cluster_id.split('_')[-1]) - 1]
        
        distances = np.linalg.norm(cluster_embeddings - cluster_center, axis=1)
        main_index = distances.argmin()
        main_sentence = cluster_df.iloc[main_index]['rating_comment']
        
        csat_impactors_list.append({
            'main_sentence': main_sentence,
            'conversation_id': cluster_df.iloc[main_index]['conversation_id'],
            'rating': rating,
            'cluster_id': cluster_id.split("_")[1],
            'type': 'rating_comment',
            'id': cluster_id
        })

csat_impactors = pd.DataFrame(csat_impactors_list)


/opt/homebrew/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/opt/homebrew/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/opt/homebrew/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


<a name="Saving-Results"></a>
## 9. Saving Results
The final step involves saving the processed data and clustering results. `user_comments` and `csat_impactors` DataFrames are saved in both CSV and newline delimited JSON formats for further analysis with BigQuery and Looker Studio.


In [ ]:
# Save conversations DataFrame to a CSV file without the index
conversations.to_csv('conversations.csv', index=False)

print("conversations saved to 'conversations.csv'")

In [84]:
# Save csat_impactors DataFrame to a newline delimited JSON file
csat_impactors.to_json('csat_impactors.json', orient='records', lines=True)

print("csat_impactors saved to 'csat_impactors.json'")

csat_impactors saved to 'csat_impactors.json'


In [88]:
# Save user_comments DataFrame to a newline delimited JSON file
user_comments.to_json('user_comments.json', orient='records', lines=True)

print("user_comments saved to 'user_comments.json'")


user_comments saved to 'user_comments.json'


In [103]:
user_comments.to_csv('user_comments.csv', index=False)